In [1]:
import sqlite3
import pandas as pd
import numpy as np

## Simple recommender system using SVD

In [3]:
conn = sqlite3.connect("OneDrive/Desktop/RestaurantRecommender/rating/recommender-project/db.sqlite3")

In [4]:
cursor = conn.cursor()

In [5]:
df = pd.read_sql_query("SELECT * FROM restaurant", conn)

In [6]:
ratings = pd.read_sql_query("SELECT * FROM restaurants_rating", conn)

In [7]:
ratings['created'] = pd.to_datetime(ratings['created'])

In [8]:
ratings = ratings.sort_values('created').drop_duplicates(['user_id', 'restaurant_id'], keep='last')

In [9]:
ratings = ratings[["user_id", "restaurant_id", "rating"]]

In [10]:
pivoted = pd.pivot_table(data=ratings, index="user_id", columns="restaurant_id", values="rating", fill_value=0, aggfunc=np.sum)

In [11]:
avg = pivoted.mean(axis=1)

In [12]:
user_item_centered = pivoted.sub(avg, axis=0)

In [13]:
from scipy.sparse.linalg import svds

In [14]:
u, e, vt = svds(user_item_centered, k=3)

In [15]:
sigma = np.diag(e)

In [16]:
recalc = np.dot(u, np.dot(sigma, vt))

In [17]:
recalc = recalc + avg.values.reshape(-1, 1)

In [18]:
recs = pd.DataFrame(recalc,
                   index=pivoted.index,
                   columns=pivoted.columns)

In [19]:
recs["user"] = recs.index

In [20]:
recs = pd.melt(recs, id_vars=["user"])

In [21]:
recs = recs.groupby(['user']).apply(lambda x: x.sort_values(by='value', ascending=False)).reset_index(drop=True)

In [28]:
new = recs.merge(ratings, how='outer', left_on=['user', 'restaurant_id'], right_on=['user_id', 'restaurant_id'], indicator=True)

In [29]:
new = new[new['_merge'] == 'left_only']

In [32]:
top3 = new.groupby(['user']).head(3)

In [34]:
top3 = top3[['user', 'restaurant_id']]

In [36]:
conn.close()